In [ ]:
import numpy as np
import pandas as pd
from simple.ohlc import renko, midPrice
from simple.plotly import chartFigure
from simple.funcs import symlog
from simple.backtest import npBacktestIOC, backtestIOC, getLong

In [ ]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
len(T)

In [ ]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]
Buy[:5]

In [ ]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

In [ ]:
len(T), len(S)

In [ ]:
C = renko(T, 0.5)
C[:5]

In [ ]:
len(C)

In [ ]:
P = pd.DataFrame(C, columns=['X0', 'X1', 'Price0', 'Price1'])
P.head()

In [ ]:
signal = np.zeros(len(S.DT))
for c in C:
    tm = T.DateTimeA[c[1]] - T.DateTimeA[c[0]]
    if tm < np.timedelta64(10000, 'μs'):
        k = np.searchsorted(S.DT, T.DateTimeA[c[1]])
        signal[k] = 1 if c[3] > c[2] else - 1
        print(c, tm, k)

In [ ]:
backtestIOC(S.DT.astype(np.int64), S.A, S.B, signal, 0)

In [ ]:
trades = npBacktestIOC(S.DT.astype(np.int64), S.A, S.B, signal, 0)
trades

In [ ]:
Long = getLong(trades)
Long

In [ ]:
trades.T0

In [ ]:
# timed chart
fig = chartFigure(
    Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3, opacity=0.25),
    Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3, opacity=0.25),
    Ask=dict(x=S.DT, y=S.A, color='red', opacity=0.2, shape='hv', mode='lines'),
    Bid=dict(x=S.DT, y=S.B, color='green', opacity=0.2, shape='hv', mode='lines'),
    Enter=dict(x=trades.T0.astype('M8[us]'), y=trades.Price0, mode='markers', 
               marker=dict(symbol='triangle-up', size=12, line_color='darkgreen', line_width=1, color='green'))                
)
fig

In [ ]:
fig.layout.shapes = [{
     'fillcolor': 'green' if c[3] > c[2] else 'red',
     'line': {'color': 'black', 'width': 2},
     'opacity': 0.15,
     'type': 'rect',
     'x0': str(T.DateTimeA[int(c[0])]),
     'x1': str(T.DateTimeA[int(c[1])]),
     'y0': c[2],
     'y1': c[3]
 } for c in C]

In [13]:
trades.T0

array([1651968067108999, 1651968076254000, 1651968091874000,
       1651968096536999, 1651968117421000], dtype=int64)

In [34]:
# timed chart
fig = chartFigure(
    Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3, opacity=0.25),
    Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3, opacity=0.25),
    Ask=dict(x=S.DT, y=S.A, color='red', opacity=0.2, shape='hv', mode='lines'),
    Bid=dict(x=S.DT, y=S.B, color='green', opacity=0.2, shape='hv', mode='lines'),
    Enter=dict(x=trades.T0.astype('M8[us]'), y=trades.Price0, mode='markers', 
               marker=dict(symbol='triangle-up', size=12, line_color='darkgreen', line_width=1, color='green'))                
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.25,
   …

In [35]:
fig.layout.shapes = [{
     'fillcolor': 'green' if c[3] > c[2] else 'red',
     'line': {'color': 'black', 'width': 2},
     'opacity': 0.15,
     'type': 'rect',
     'x0': str(T.DateTimeA[int(c[0])]),
     'x1': str(T.DateTimeA[int(c[1])]),
     'y0': c[2],
     'y1': c[3]
 } for c in C]